In [1]:

import time
import math
import json
import pickle

import statistics

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt

from os import listdir
from os.path import join
from os.path import isfile

from itertools import combinations

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


from libsvm.svmutil import svm_problem
from libsvm.svmutil import svm_parameter
from libsvm.svmutil import svm_train
from libsvm.svmutil import svm_predict
from libsvm.svmutil import evaluations

from Function import svm_function

In [15]:
perf_list = []

In [18]:
# dir_path = "data/merge_data/output/"
dir_path = "data/RBP-PWM/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]


for file_name in onlyfiles:
    if ("__esvm_p10_f10_s10__v230221.json" in file_name) and not ".dvc" in file_name:
        file_str = file_name[:-len("__esvm_p10_f10_s10__v230221.json")]
    # if ("__esvm_p10_f10_s10__v230118_v2.json" in file_name) and not ".dvc" in file_name:
    #     file_str = file_name[:-len("__esvm_p10_f10_s10__v230118_v2.json")]
        
        if "train__" in file_str:
            file_str = file_str.replace("train", "train_binary")
        print(file_str)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        num = np.load("data/RBP-PWM/train/" + file_str + "__x.npy").shape[1]
        perf_list.append([file_str, num,
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC'],
                          ])

train_binary__10-4__filter
train_binary__10-3__filter
train_-log10__10-3__filter
train_-log10__10-4__filter


In [19]:
perf_list

[['train_binary__10-6',
  108,
  0.8666025842613891,
  0.00629700159722099,
  0.9954159328706101,
  0.007364676950540675,
  0.8697832292545771,
  0.0023062007655108383,
  0.016781609195402298,
  0.016784758010364883,
  nan,
  nan,
  0.9283550448724187,
  0.00358880597901085,
  0.5078366680122075,
  0.009350191303514223],
 ['train_log2__10-5',
  108,
  0.6534879298619504,
  0.2796701641894262,
  0.7020071480368796,
  0.38422123239893463,
  0.8925899010072221,
  0.03583240574948732,
  0.3306896551724138,
  0.41017641993215287,
  0.1564467973890985,
  0.05111237916238267,
  0.6987195899313308,
  0.32303987213143925,
  0.5383041169382159,
  0.033919619449358646],
 ['train_-log10__10-4',
  108,
  0.49512496372244535,
  0.041169685243831335,
  0.46233813322283224,
  0.04403972208264886,
  0.9127131172801345,
  0.020459869090247777,
  0.7111494252873565,
  0.06126997451216082,
  0.1677550130859618,
  0.018747258036940776,
  0.6128130119069648,
  0.04271325466827676,
  0.6070912666659523,
  0.

In [20]:
pd.DataFrame(perf_list, columns=["esvm_s10", "nums", 
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ]).sort_values("esvm_s10").to_csv("data/RBP-PWM/output/esvm_s10_f10_p10__v230306__pref.csv", index=False)

In [5]:
np.load("data/linear_features/point/benchmark/train/k234p10nor2n3_train.npy").shape

(2264, 1008)

In [3]:
f = open("data/merge_data/output/Kmer_k3__DACC_lag2__SC-PseDNC-General_all_index_svmf10p4e512.json")
perf = json.load(f)
f.close()
["file_str", 
        perf['avg Accy'], perf['std Accy'],
        perf['avg Recall'], perf['std Recall'],
        perf['avg Prec'], perf['std Prec'],
        perf['avg Spec'], perf['std Spec'],
        perf['avg F1sc'], perf['std F1sc'],
        perf['avg AUROC'], perf['std AUROC'],
        ]

['file_str',
 0.7062645551932929,
 0.02728313642700328,
 0.724779861183052,
 0.027373539164752204,
 0.9191811524505985,
 0.013461697940586836,
 0.585483870967742,
 0.0729476979210487,
 0.8102467384324458,
 0.019711847903634495,
 0.6936555352450462,
 0.04558923440965074]

In [16]:
from numpy import average


dir_path = "data/linear_features/point/benchmark/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

perf_list = []

for file_name in onlyfiles:
    if "_svmf10p4e512.json" in file_name and not ".dvc" in file_name and not "train" in file_name:
        
        file_str = file_name[:-5]
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        perf_list.append([file_str, np.load("data/linear_features/point/benchmark/train/" + file_name[:-18] + "_train.npy").shape[1],
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC'],
                          ])
        # npv_array = []
        # for i in range(len(perf['confusion matrix'])):
        #     tn, fp, fn, tp = np.array(perf['confusion matrix'][i][1]).flatten()
        #     acc = (tn + tp) / (tn + fp + fn + tp)
        #     recall = tp / (fn + tp)
        #     prec = (tp / (fp + tp))
        #     spec = (tn / (tn + fp))
        #     npv = (tn / (tn + fn))
        #     f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
        #     npv_array.append(npv)
        # print(npv_array)
        # print(sum(npv_array)/ len(npv_array), np.std(npv_array))

In [18]:
pd.DataFrame(perf_list, columns=['file name', "num", 'avg Accy', 'std Accy', 'avg Recall', 'std Recall', 'avg Prec', 'std Prec', 'avg Spec', 'std Spec', 'avg F1sc', 'std F1sc', 'avg AUROC', 'std AUROC']).sort_values('file name').to_csv(dir_path + "svmf10p4e512_pref_v2.csv", index=False)